# Framenet in NLTK

(C) 2019-2024 by [Damir Cavar](http://damir.cavar.me/)

**Version:** 1.2, January 2024

**License:** [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))

**Prerequisites:**

In [ ]:
!pip install -U nltk

The material in this notebook is based on:
- [The NLKT Howto on Framenet](https://www.nltk.org/howto/framenet.html)
- [The Framenet Website](https://framenet.icsi.berkeley.edu/fndrupal/)


The FrameNet corpus is a lexical database of English that is both human- and machine-readable, based on annotating examples of how words are used in actual texts. FrameNet is based on a theory of meaning called Frame Semantics, deriving from the work of Charles J. Fillmore and colleagues. The basic idea is straightforward: that the meanings of most words can best be understood on the basis of a semantic frame: a description of a type of event, relation, or entity and the participants in it. For example, the concept of cooking typically involves a person doing the cooking (Cook), the food that is to be cooked (Food), something to hold the food while cooking (Container) and a source of heat (Heating_instrument). In the FrameNet project, this is represented as a frame called Apply_heat, and the Cook, Food, Heating_instrument and Container are called frame elements (FEs). Words that evoke this frame, such as fry, bake, boil, and broil, are called lexical units (LUs) of the Apply_heat frame. The job of FrameNet is to define the frames and to annotate sentences to show how the FEs fit syntactically around the word that evokes the frame.

## Frames

A Frame is a script-like conceptual structure that describes a particular type of situation, object, or event along with the participants and props that are needed for that Frame. For example, the "Apply_heat" frame describes a common situation involving a Cook, some Food, and a Heating_Instrument, and is evoked by words such as bake, blanch, boil, broil, brown, simmer, steam, etc.

We call the roles of a Frame "frame elements" (FEs) and the frame-evoking words are called "lexical units" (LUs).

FrameNet includes relations between Frames. Several types of relations are defined, of which the most important are:
- Inheritance: An IS-A relation. The child frame is a subtype of the parent frame, and each FE in the parent is bound to a corresponding FE in the child. An example is the "Revenge" frame which inherits from the "Rewards_and_punishments" frame.
- Using: The child frame presupposes the parent frame as background, e.g the "Speed" frame "uses" (or presupposes) the "Motion" frame; however, not all parent FEs need to be bound to child FEs.
- Subframe: The child frame is a subevent of a complex event represented by the parent, e.g. the "Criminal_process" frame has subframes of "Arrest", "Arraignment", "Trial", and "Sentencing".
- Perspective_on: The child frame provides a particular perspective on an un-perspectivized parent frame. A pair of examples consists of the "Hiring" and "Get_a_job" frames, which perspectivize the "Employment_start" frame from the Employer's and the Employee's point of view, respectively.

To get a list of all of the Frames in FrameNet, you can use the frames() function. If you supply a regular expression pattern to the frames() function, you will get a list of all Frames whose names match that pattern:

In [1]:
from pprint import pprint

In [2]:
from nltk.corpus import framenet as fn

In [3]:
len(fn.frames())

1221

In [4]:
pprint(fn.frames(r'(?i)medical'))

[<frame ID=239 name=Medical_conditions>, <frame ID=257 name=Medical_instruments>, ...]


To get the details of a particular Frame, you can use the frame() function passing in the frame number:

In [5]:
f = fn.frame(256)

In [6]:
f.ID

256

In [7]:
f.name

'Medical_specialties'

In [8]:
f.definition

"This frame includes words that name medical specialties and is closely related to the Medical_professionals frame.  The FE Type characterizing a sub-are in a Specialty may also be expressed. 'Ralph practices paediatric oncology.'"

In [9]:
len(f.lexUnit)

29

In [10]:
pprint(sorted([x for x in f.FE]))

['Affliction', 'Body_system', 'Specialty', 'Type']


In [11]:
pprint(f.frameRelations)

[<Parent=Medical_interaction_scenario -- Using -> Child=Medical_specialties>]


The frame() function shown above returns a dict object containing detailed information about the Frame. See the documentation on the frame() function for the specifics.

You can also search for Frames by their Lexical Units (LUs). The frames_by_lemma() function returns a list of all frames that contain LUs in which the 'name' attribute of the LU matchs the given regular expression. Note that LU names are composed of "lemma.POS", where the "lemma" part can be made up of either a single lexeme (e.g. 'run') or multiple lexemes (e.g. 'a little') (see below).

In [12]:
for x in fn.frames_by_lemma(r'(?i)a little'):
    print(x)

frame (2001): Degree

[URL] https://framenet2.icsi.berkeley.edu/fnReports/data/frame/Degree.xml

[definition]
  LUs in this frame modify a gradable attribute and describe
  intensities at the extreme positions on a scale.  'She is very
  brave.'  'It is that simple!'  'The feuds are so vicious because
  the stakes are so low.'  'The plant is totally red  without the
  slightest hint of green.'

[semTypes] 1 semantic types
  End_of_scale(98)

[frameRelations] 2 frame relations
  <Parent=Importance -- Using -> Child=Degree>
  <MainEntry=Hedging -- See_also -> ReferringEntry=Degree>

[lexUnit] 21 lexical units
  a little bit.adv (14744), a little.adv (14743), a lot.adv
  (17659), as hell.adv (18815), crystal (clear).n (18813), dirt
  (poor).n (18811), far.adv (14978), fully.adv (14753), great.a
  (16311), heavily.adv (14826), in part.adv (16165), kind of.adv
  (18816), particular.a (16667), so.adv (14742), somewhat.adv
  (15970), sopping (wet).a (18812), stone (cold).n (18810),
  that.adv

## Lexical Units

A lexical unit (LU) is a pairing of a word with a meaning. For example, the "Apply_heat" Frame describes a common situation involving a Cook, some Food, and a Heating Instrument, and is _evoked_ by words such as bake, blanch, boil, broil, brown, simmer, steam, etc. These frame-evoking words are the LUs in the Apply_heat frame. Each sense of a polysemous word is a different LU.

We have used the word "word" in talking about LUs. The reality is actually rather complex. When we say that the word "bake" is polysemous, we mean that the lemma "bake.v" (which has the word-forms "bake", "bakes", "baked", and "baking") is linked to three different frames:
- Apply_heat: "Michelle baked the potatoes for 45 minutes."
- Cooking_creation: "Michelle baked her mother a cake for her birthday."
- Absorb_heat: "The potatoes have to bake for more than 30 minutes."

These constitute three different LUs, with different definitions.

Multiword expressions such as "given name" and hyphenated words like "shut-eye" can also be LUs. Idiomatic phrases such as "middle of nowhere" and "give the slip (to)" are also defined as LUs in the appropriate frames ("Isolated_places" and "Evading", respectively), and their internal structure is not analyzed.

Framenet provides multiple annotated examples of each sense of a word (i.e. each LU). Moreover, the set of examples (approximately 20 per LU) illustrates all of the combinatorial possibilities of the lexical unit.

Each LU is linked to a Frame, and hence to the other words which evoke that Frame. This makes the FrameNet database similar to a thesaurus, grouping together semantically similar words.

In the simplest case, frame-evoking words are verbs such as "fried" in:
- "Matilde fried the catfish in a heavy iron skillet."

Sometimes event nouns may evoke a Frame. For example, "reduction" evokes "Cause_change_of_scalar_position" in:
- "...the reduction of debt levels to $665 million from $2.6 billion."

Adjectives may also evoke a Frame. For example, "asleep" may evoke the "Sleep" frame as in:
- "They were asleep for hours."

Many common nouns, such as artifacts like "hat" or "tower", typically serve as dependents rather than clearly evoking their own frames.

Details for a specific lexical unit can be obtained using this class's lus() function, which takes an optional regular expression pattern that will be matched against the name of the lexical unit:

In [13]:
len(fn.lus())

13572

In [14]:
pprint(fn.lus(r'(?i)a little'))

[<lu ID=14744 name=a little bit.adv>, <lu ID=14743 name=a little.adv>, ...]


You can obtain detailed information on a particular LU by calling the lu() function and passing in an LU's 'ID' number:

In [15]:
fn.lu(256).name

'foresee.v'

In [16]:
fn.lu(256).definition

'COD: be aware of beforehand; predict.'

In [17]:
fn.lu(256).frame.name

'Expectation'

In [18]:
fn.lu(255).lexemes[0].name
for x in fn.lu(255).lexemes:
    print(x.name)

expect


Note that LU names take the form of a dotted string (e.g. "run.v" or "a little.adv") in which a lemma preceeds the "." and a part of speech (POS) follows the dot. The lemma may be composed of a single lexeme (e.g. "run") or of multiple lexemes (e.g. "a little"). The list of POSs used in the LUs is:
- v - verb
- n - noun
- a - adjective
- adv - adverb
- prep - preposition
- num - numbers
- intj - interjection
- art - article
- c - conjunction
- scon - subordinating conjunction

For more detailed information about the info that is contained in the dict that is returned by the lu() function, see the documentation on the lu() function.

## Annotated Documents

The FrameNet corpus contains a small set of annotated documents. A list of these documents can be obtained by calling the documents() function:

In [19]:
docs = fn.docs()

In [20]:
len(docs)

107

In [21]:
pprint(sorted(docs[0].keys()))
pprint(docs[0].sentence)

['ID',
 'URL',
 '_type',
 'corpid',
 'corpname',
 'description',
 'filename',
 'name',
 'sentence']
[full-text sentence (4154255) in chapter8_911report:


[POS] 6 tags

[POS_tagset] PENN

[text] + [annotationSet]

'' THE SYSTEM WAS BLINKING RED"
 
 
 

,
 full-text sentence (4154256) in chapter8_911report:


[POS] 4 tags

[POS_tagset] PENN

[text] + [annotationSet]

THE SUMMER OF THREAT
 
 
 

,
 full-text sentence (4154257) in chapter8_911report:


[POS] 11 tags

[POS_tagset] PENN

[text] + [annotationSet]

As 2001 began , counterterrorism officials were receiving 
 
 
 

frequent but fragmentary
 
 
 

,
 full-text sentence (4154258) in chapter8_911report:


[POS] 4 tags

[POS_tagset] PENN

[text] + [annotationSet]

reports about threats .
 
 
 

,
 full-text sentence (4154259) in chapter8_911report:


[POS] 9 tags

[POS_tagset] PENN

[text] + [annotationSet]

Indeed , there appeared to be possible threats almost
 
 
 

,
 full-text sentence (4154260) in chapter8_911report:


[POS] 9

Detailed information about each sentence contained in each document can be obtained by calling the annotated_document() function and supplying the 'ID' number of the document. For detailed information about the info that is for each document, see the documentation on the annotated_document() function.